In [ ]:
import os
import pandas as pd
from datetime import datetime
import locale

locale.setlocale(locale.LC_TIME,'pt_BR.UTF-8')

pasta = r'D:\PJES_AGOSTO_24'

# Caminho do log de erros
log_erros = os.path.join(pasta,f"log_erros_{datetime.now().strftime('%Y%m%d_%H%M%S')}.txt")

# Funcao para registra erros no arquivo
def registrar_erro(mensagem):
    with open(log_erros, 'a',encoding='utf-8')as f:
        f.write(f"{datetime.now().strftime('%Y-%m-%d %H:%M:%S')} - {mensagem}\n")

# Lista de cargos cota de 300
oficiais = [c.title() for c in ["Oficial da PM", "Tenente", "Capitão", "Major", 
            "Tenente Coronel", "Coronel","Delegado de Policia",
            "Perito Criminal","Medico Legista"]]


#Funcao para calcular verba
def calcular_verba(data_fim, competencia_mes):
    if pd.isna(data_fim):
        return ""
    if data_fim.month == competencia_mes.month and data_fim.year == competencia_mes.year:
        return 223
    elif data_fim < competencia_mes:
        return 423
    else:
        return""

def calcular_valor(row):
    try:
        cargo = str(row['CARGO']).strip().title()
        qtd_cota = float(row['QTDCOTA'])
        if cargo in oficiais:
            return qtd_cota * 300
        else:
            return qtd_cota * 200
    except Exception as e:
        print(f"Erro ao calcular valor para linha: {row}")
        print(e)
        return 0
# Data de referencia da competecia
competencia_data = datetime(2024, 9, 1)

# lista de armazenamento dos dataframes
dfs = []

for arquivo in os.listdir(pasta):
    if arquivo.endswith('.xlsx'):
        caminho_arquivo = os.path.join(pasta, arquivo)
        try:
            df = pd.read_excel(caminho_arquivo,
                               sheet_name = 'Planilha de PJES',
                               skiprows=9,
                               usecols = "A:L"
            )
            df.dropna(how='all',inplace=True)
            df.columns = df.columns.str.upper()
            
            #Tratamentos
            df['DT TERMINO'] = pd.to_datetime(df['DT TERMINO'], errors='coerce', dayfirst=True)
            df['CARGO'] = df['CARGO'].str.strip().str.title()
            df['QTDCOTA'] = pd.to_numeric(df['QTDCOTA'], errors='coerce').fillna(0)
            df['VALOR'] = df.apply(calcular_valor, axis=1)


            #Novas Colunas conforme o padrao
            df['EXERCÍCIO'] = 2024
            df['COMPETÊNCIA'] = competencia_data.strftime('%B').upper()
            df['ORIGEM COTA'] = 'DECRETO'
            df['ORIGEM DA INFORMAÇÃO'] = os.path.basename(arquivo).replace('.xlsx','')
            df['OPERATIVA QUE PRESTOU SERVIÇO'] = df['OPERATIVA']
            df['LOCAL DA PRESTAÇÃO DO SERVIÇO'] = df['TITULO']
            df['COTA'] = df['QTDCOTA'].astype(int)
            df['VERBA'] = df['DT TERMINO'].apply(lambda x: calcular_verba(x, competencia_data))
            df['CARGO'] = df['CARGO'].str.strip().str.title()
            df['SITUAÇÃO'] = 'ATIVO'
            df['MATRICULA'] = df['MATRICULA'].astype(str).str.replace(r'\.0$','',regex=True)
            df['NOME'] = df['NOME EXTENSO']
            df['VALOR'] = df['VALOR']

            # Organizar colunas 
            colunas_finais = [
                'EXERCÍCIO', 'COMPETÊNCIA', 'ORIGEM COTA', 'ORIGEM DA INFORMAÇÃO',
                'OPERATIVA QUE PRESTOU SERVIÇO', 'LOCAL DA PRESTAÇÃO DO SERVIÇO',
                'MATRICULA', 'NOME', 'COTA', 'VERBA', 'CARGO', 'SITUAÇÃO','VALOR'
            ]
            df_modificado = df[colunas_finais].copy()

            # Agrupando por matricula dentro de cada planilha
            colunas_agrupamento = [
                'EXERCÍCIO', 'COMPETÊNCIA', 'ORIGEM COTA', 'ORIGEM DA INFORMAÇÃO',
                'OPERATIVA QUE PRESTOU SERVIÇO', 'LOCAL DA PRESTAÇÃO DO SERVIÇO',
                'MATRICULA', 'NOME', 'VERBA', 'CARGO', 'SITUAÇÃO'
                ]
                
            df_agrupado_por_planilha = df_modificado.groupby(colunas_agrupamento, as_index=False).agg({
                            'COTA': 'sum',
                            'VALOR': 'sum'
                            }).sort_values(by='COTA', ascending=False)
            
            dfs.append(df_agrupado_por_planilha)

        except Exception as e:
                msg_erro = f"Erro ao processar o arquivo '{arquivo}': {str(e)}"
                print("⚠️", msg_erro)
                registrar_erro(msg_erro)
if dfs:
    data =pd.concat(dfs, ignore_index=True)
    data = data.sort_values(by='COTA',ascending=False)

    #Exibir a base consolidada antes da tabela dinamica
    print("\n📋 Base consolidada por matrícula (antes da Tabela Dinâmica):\n")
    print(data.head(10))

    # Converter COTA para número novamente para segurança
    data['COTA'] = pd.to_numeric(data['COTA'], errors='coerce').fillna(0)

    # Criar tabela dinamica
    tabela_pivot = pd.pivot_table(
        data,
        index=['MATRICULA','NOME'],
        columns='LOCAL DA PRESTAÇÃO DO SERVIÇO',
        values='COTA',
        aggfunc='sum',
        fill_value=0
    ).reset_index()

    tabela_pivot.columns.name=None # remove o nome das colunas herdado do columns=
    tabela_pivot['TOTAL GERAL']=tabela_pivot.iloc[:,2:].sum(axis=1)
    tabela_pivot = tabela_pivot.sort_values(by='TOTAL GERAL',ascending= False)

    # Tabela por verba

    tabela_por_verba = data.groupby(['MATRICULA', 'NOME','VERBA'], as_index=False).agg({
        'COTA': 'sum',
        'VALOR': 'sum'
    }).sort_values(by=['MATRICULA','VERBA'],ascending=False)

    # verba 223
    tabela_verba_223 = data[data['VERBA'].astype(str) == '223'].copy()
    tabela_verba_223 = tabela_verba_223.sort_values(by='COTA', ascending=False)

    # Nome do arquivo final
    agora = datetime.now().strftime("%Y%m%d_%H%M%S")
    nome_saida = f"TABELA_BASE_{agora}.xlsx"
    caminho_saida = os.path.join(pasta, nome_saida)
    nome_aba_mes = competencia_data.strftime('%B').upper()
    
    # Salvar em Excel
    with pd.ExcelWriter(caminho_saida, engine='openpyxl') as writer:
        data.to_excel(writer, index=False, sheet_name=nome_aba_mes)  # <-- Nome dinâmico
        tabela_pivot.to_excel(writer, index=False, sheet_name='RESUMO POR LOCAL')
        tabela_verba_223.to_excel(writer, index=False, sheet_name='VERBA 223') 
        tabela_por_verba.to_excel(writer, index=False, sheet_name='TOTAL POR VERBA')

    print(f"\n✅ Arquivo salvo com sucesso em:\n{caminho_saida}")

    # Excluir log se estiver vazio
    if os.path.exists(log_erros) and os.path.getsize(log_erros) == 0:
        os.remove(log_erros)

else:
    print("Nenhum dado processado.")

   